This notebook's objective is to scrap product information from Amazon's brazilian e-commerce and create a tabular structure for later use in product categorization. We'll focus on four major product categories related to  Eletronics, Computers and Informatics.

Final output is a .csv file

In [ ]:
# libs
import numpy as np
import pandas as pd
import itertools 
import requests
import time
from bs4 import BeautifulSoup
from random import seed
from random import random
from random import randint
import re

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [ ]:
# loads a .csv file with the link to the webpages
urls = pd.read_csv('LINKS2.csv', delimiter =";",encoding='latin-1')

In [ ]:
urls

,Departamento,URL
0,Computadores e Informática,https://www.amazon.com.br/s?i=computers&rh=n%3...
1,Eletrodomésticos,https://www.amazon.com.br/s?i=appliances&rh=n%...
2,Eletroportáteis,https://www.amazon.com.br/s?i=kitchen&rh=n%3A1...


In [ ]:
# prints links for reference
for j in range(len(urls)):
    for i in range(2):
        url = urls['URL'][j]
        index = url.find('page=')+5
        url_final = url[:index] + str(i) + url[index+1:-1]+str(i)
        #print(url[:-1]+str(i))
        print(url_final)
        
#s[:index] + newstring + s[index + 1:]

https://www.amazon.com.br/s?i=electronics&rh=n%3A16243803011&fs=true&page=0&qid=1621732810&ref=sr_pg_0
https://www.amazon.com.br/s?i=electronics&rh=n%3A16243803011&fs=true&page=1&qid=1621732810&ref=sr_pg_1
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=0&qid=1621732751&ref=sr_pg_0
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=1&qid=1621732751&ref=sr_pg_1
https://www.amazon.com.br/s?i=appliances&rh=n%3A16522082011&fs=true&page=0&qid=1621733127&ref=sr_pg_0
https://www.amazon.com.br/s?i=appliances&rh=n%3A16522082011&fs=true&page=1&qid=1621733127&ref=sr_pg_1
https://www.amazon.com.br/s?i=kitchen&rh=n%3A17124722011&fs=true&page=0&qid=1621733241&ref=sr_pg_0
https://www.amazon.com.br/s?i=kitchen&rh=n%3A17124722011&fs=true&page=1&qid=1621733241&ref=sr_pg_1


In [ ]:
proxies_list = [['http://186.209.33.154:8080'],
               ['http://189.127.229.92:8080'],
               ['http://169.57.157.146:8123'],
               ['http://201.91.82.155:3128']]

user_agent = [
        ['Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246'],
        ['Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1'],
        ['Mozilla/5.0 (Linux; U; Android 4.2.3; he-il; NEO-X5-116A Build/JDQ39) AppleWebKit/534.30 (KHTML, like Gecko) Version/5.0 Safari/534.30'],
        ['Mozilla/5.0 (X11; U; Linux armv7l like Android; en-us) AppleWebKit/531.2+ (KHTML, like Gecko) Version/5.0 Safari/533.2+ Kindle/3.0+'],
        ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'],
        ['Mozilla/5.0 (iPhone; CPU iPhone OS 12_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/69.0.3497.105 Mobile/15E148 Safari/605.1']
        ]


data =         {"Departamento":[],
                "produto":[],
                "preco_inteiro":[],
                "preco_decimal":[],
                "prazo":[],
                "frete":[],
                "qtd_reviews":[],
                "rating":[],
                "tag":[],
                "url_produto":[]}


for j in range(len(urls)):
    for i in range(20):
        url = urls['URL'][j]
        index = url.find('page=')+5
        url_final = url[:index] + str(i) + url[index+1:-1]+str(i)
        #print(url[:-1]+str(i))
        print(url_final)
        #for j in range(len(URL)):
        str_user = ','.join(user_agent[ randint(0,7)])
        headers = {
            'User-Agent': str_user}
        #print(str_user)
        proxies_rand = ''.join(proxies_list[randint(0,3)])
        #print(proxies_rand)
        proxy = {"http":proxies_rand}
        #print(urls['URL'][j])
        #print(urls[j])
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        response = requests.get(url_final, headers=headers, proxies = proxy,timeout=5)
    #    response = requests.get(URL[j], headers=headers, proxies = proxy,timeout=5)
    #    print(URL[j])  
        content = response.content
        parser = BeautifulSoup(content, 'html.parser')
        box = parser.find_all(class_="a-section a-spacing-medium")
        #Checa se o padrão do box mudou
        if not box:
            box = parser.find_all(class_="s-expand-height s-include-content-margin s-border-bottom s-latency-cf-section")
            if not box:
                box = parser.find_all(class_="s-card-container s-overflow-hidden s-include-content-margin s-latency-cf-section s-card-border")      
        for i in range(len(box)):
            try:
                data["Departamento"].append(urls['Departamento'][j])
                try:
                    data['produto'].append(box[i].find(class_="a-size-base-plus a-color-base a-text-normal").get_text(strip=True))    
                except:
                    try:
                        data['produto'].append(box[i].find(class_="a-size-medium a-color-base a-text-normal").get_text(strip=True)) 
                    except:    
                        try:
                            data['produto'].append(box[i].find(class_="a-size-small a-color-base a-text-normal").get_text(strip=True)) 
                        except:
                            data['produto'].append(None) 
                try:
                    data['preco_inteiro'].append(box[i].find(class_="a-price-whole").get_text(strip=True))
                except:
                    data['preco_inteiro'].append(None)
                try:
                    data['preco_decimal'].append(box[i].find(class_="a-price-fraction").get_text(strip=True))
                except:
                    data['preco_decimal'].append(None)
                try:
                    data['frete'].append(box[i].find(class_="a-row a-size-base a-color-secondary s-align-children-center").get_text(strip=True))
                except:
                    data['frete'].append(None)
                try:
                    data['qtd_reviews'].append(box[i].select('.a-size-small > :nth-child(2)')[0].get_text(strip=True))
                except:
                    data['qtd_reviews'].append(None)
                try:
                    data['rating'].append(box[i].select('i', {'class': 'acr-average-stars-rating-text'})[0].get_text(strip=True))
                except:
                    data['rating'].append(None)
                try:
                    data['tag'].append(box[i].find_all(class_="a-badge-text"))
                except:
                    data['tag'].append(None)
                try:
                    if len(box[i].select('.sg-row:nth-last-child(3):nth-child(2) > .sg-col-4-of-20 > DIV.sg-col-inner:nth-child(1):nth-last-child(1) > .a-spacing-top-micro > :nth-child(2)')[0].get_text(strip=True))!=1:
                        data['prazo'].append(box[i].select('.sg-row:nth-last-child(3):nth-child(2) > .sg-col-4-of-20 > DIV.sg-col-inner:nth-child(1):nth-last-child(1) > .a-spacing-top-micro > :nth-child(2)')[0].get_text(strip=True)) 
                    else:
                        data['prazo'].append(None)  
                except:
                    data['prazo'].append(None)
                try:
                    data['url_produto'].append(box[i].find(class_="a-link-normal a-text-normal")['href'])
                except:
                    data['url_produto'].append(None)       

            except:
                print("Erro")
                continue
        value = random()
        scaled_value = 1 + (value * (9 - 1))
        time.sleep(scaled_value)

https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=0&qid=1621732751&ref=sr_pg_0
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=1&qid=1621732751&ref=sr_pg_1
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=2&qid=1621732751&ref=sr_pg_2
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=3&qid=1621732751&ref=sr_pg_3
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=4&qid=1621732751&ref=sr_pg_4
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=5&qid=1621732751&ref=sr_pg_5
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=6&qid=1621732751&ref=sr_pg_6
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=7&qid=1621732751&ref=sr_pg_7
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=8&qid=1621732751&ref=sr_pg_8
https://www.amazon.com.br/s?i=computers&rh=n%3A16339926011&fs=true&page=9&qid=1621732751&re

In [ ]:
# transforms collected data into a Pandas DataFrame
df = pd.DataFrame(data)
df

,Departamento,produto,preco_inteiro,preco_decimal,prazo,frete,qtd_reviews,rating,tag,url_produto
0,Computadores e Informática,Notebook Lenovo Ultrafino ideapad S145 i5-1035...,"3.699,",00,None,None,None,"4,4 de 5 estrelas",[[Mais vendido]],None
1,Computadores e Informática,Impressora multifuncional HP DeskJet Ink Advan...,"378,",00,None,None,None,"4,6 de 5 estrelas",[],None
2,Computadores e Informática,Multifuncional Epson EcoTank L3150 - Tanque de...,"1.109,",00,None,None,None,"4,8 de 5 estrelas",[[Mais vendido]],None
3,Computadores e Informática,"Suporte para Notebook, OCTOO, Uptable, UP-BL, ...","45,",90,None,None,None,"4,4 de 5 estrelas",[[Mais vendido]],None
4,Computadores e Informática,"Apple iPad 8ª Geração 10.2"", Wi-Fi, 128GB Spac...","3.378,",99,None,None,None,"4,8 de 5 estrelas",[],None
...,...,...,...,...,...,...,...,...,...,...
983,Eletroportáteis,Cafeteira Elétrica Smart Coffe 2 Xícaras 127V ...,"132,",90,None,"Receba atéSexta-feira, 18 de junFrete GRÁTIS n...",21,"4,3 de 5 estrelas",[],/Cafeteira-El%C3%A9trica-Smart-Coffe-X%C3%ADca...
984,Eletroportáteis,"Sanduicheira Waffle Chrome, Preta, 110v, Oster","359,",90,None,"R$ 17,08 de frete",143,"4,3 de 5 estrelas",[],/Sanduicheira-Waffle-Oster-CKSTSM3892-017-Prat...
985,Eletroportáteis,Mini Processador de Alimentos 127V com 100W 35...,"109,",90,None,"R$ 17,74 de frete",52,"4,3 de 5 estrelas",[],/Processador-Alimentos-350ml-L%C3%A2minas-Mult...
986,Eletroportáteis,Sanduicheira Agratto Inox 2 Sid 01-127V 750W A...,"86,",23,None,"Receba atéamanhã, 5 de junFrete GRÁTIS no seu ...",92,"4,3 de 5 estrelas",[],/Sanduicheira-Grill-750w-Inox-127v/dp/B07QMFGR...


In [ ]:
# a few adjustments to get data in proper state

# safe copy
df_clean = df.copy()

# price variable
df_clean['preco'] = df_clean['preco_inteiro']+df_clean['preco_decimal']
df_clean['preco'] = [str(x).replace('.','') for x in df_clean["preco"]]
df_clean['preco'] = [str(x).replace(',','.') for x in df_clean["preco"]]
df_clean['preco'] = df_clean['preco'].apply(lambda x: float(x))

# rating (=user evaluation) variable
df_clean['rating_ajustado'] = df_clean['rating'].str[:3].replace(',', '.',regex=True)

# freight variable
df_clean['frete_gratis'] = df_clean['frete'].str.lower()
df_clean['frete_gratis'] = df_clean['frete_gratis'].apply(lambda row: row[row.find('frete grátis'):row.find('frete grátis')+12] if pd.notnull(row) else row)

# delivery deadline variable
df_clean['prazo_ajustado'] = df_clean['frete'].str.lower()
df_clean['prazo_ajustado'] = df_clean['prazo_ajustado'].apply(lambda row : re.sub("[^0-9]", "", row[row.find('até'):row.find('até')+18]) if pd.notnull(row) else row )
dia_coleta = 4
df_clean['prazo_ajustado'] = df_clean['prazo_ajustado'].apply(pd.to_numeric)
df_clean['prazo_ajustado'] = df_clean['prazo_ajustado'].apply(lambda row : np.abs(row-dia_coleta) if (row>dia_coleta) else (np.abs((30-dia_coleta) + row) if (dia_coleta<row) else row))

# Where there is a price for freight, take the string and turn it into numeric
df_clean['frete_preco'] = df_clean['frete'].loc[df_clean['frete'].str.find('R$')==0].apply(lambda x: pd.to_numeric(x[3:x.find(' de frete')].replace(',','.')))
# Where an option for free frete is available, put a zero
df_clean['frete_preco'][df_clean['frete_gratis']=='frete grátis'] = 0
df_clean['frete_gratis_binario']=0
df_clean['frete_gratis_binario'].loc[df_clean['frete_gratis']== 'frete grátis'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df_clean.head()

,Departamento,produto,preco_inteiro,preco_decimal,prazo,frete,qtd_reviews,rating,tag,url_produto,preco,rating_ajustado,frete_gratis,prazo_ajustado,frete_preco,frete_gratis_binario
0,Computadores e Informática,Notebook Lenovo Ultrafino ideapad S145 i5-1035...,"3.699,",00,None,None,None,"4,4 de 5 estrelas",[[Mais vendido]],None,3699.00,4.4,None,NaN,NaN,0
1,Computadores e Informática,Impressora multifuncional HP DeskJet Ink Advan...,"378,",00,None,None,None,"4,6 de 5 estrelas",[],None,378.00,4.6,None,NaN,NaN,0
2,Computadores e Informática,Multifuncional Epson EcoTank L3150 - Tanque de...,"1.109,",00,None,None,None,"4,8 de 5 estrelas",[[Mais vendido]],None,1109.00,4.8,None,NaN,NaN,0
3,Computadores e Informática,"Suporte para Notebook, OCTOO, Uptable, UP-BL, ...","45,",90,None,None,None,"4,4 de 5 estrelas",[[Mais vendido]],None,45.90,4.4,None,NaN,NaN,0
4,Computadores e Informática,"Apple iPad 8ª Geração 10.2"", Wi-Fi, 128GB Spac...","3.378,",99,None,None,None,"4,8 de 5 estrelas",[],None,3378.99,4.8,None,NaN,NaN,0


In [ ]:
df_clean.to_csv('/content/data.csv")